In [8]:
import pandas as pd

# Assuming you have already loaded your data into a DataFrame named df
# For example: df = pd.read_json('/content/DataEngineeringQ2.json')

# Sample data for demonstration (replace this with your actual data)
data = {
    'firstName': ['Dinesh', 'Raghu', 'Daniya', 'jay', '', ''],
    'lastName': ['Kumar', 'Viju', '', 'Zehra', '', 'Sharma'],
    'DOB': ['1983-05-16', '', '2001-05-16', '', '1998-11-19', '']
}
df = pd.DataFrame(data)

# Define a function to calculate the percentage of missing values
def calculate_missing_percentage(column):
    total = len(df[column])
    missing = df[column].apply(lambda x: x == '' or pd.isna(x)).sum()
    return (missing / total) * 100

# Columns to check
columns = ['firstName', 'lastName', 'DOB']

# Calculate and print the percentages
for column in columns:
    missing_percentage = calculate_missing_percentage(column)
    print(f"{column}: {missing_percentage:.2f}%")

# If you have your DataFrame named `patientDetails`, replace `df` with `patientDetails`


firstName: 33.33%
lastName: 33.33%
DOB: 50.00%


In [19]:
import json

# Load the JSON file
input_filename = '/content/DataEngineeringQ2 (1).json'  # Replace with your actual file name
output_filename = 'cleaned_data.json'  # Name for the output file

# Function to remove null values from JSON data
def remove_nulls(data):
    if isinstance(data, dict):
        # Remove null values from dictionary
        return {k: remove_nulls(v) for k, v in data.items() if v is not None}
    elif isinstance(data, list):
        # Remove null values from list
        return [remove_nulls(item) for item in data if item is not None]
    else:
        return data

# Read the JSON file
with open(input_filename, 'r') as file:
    json_data = json.load(file)

# Remove null values
cleaned_data = remove_nulls(json_data)

# Save the cleaned JSON data to a new file
with open(output_filename, 'w') as file:
    json.dump(cleaned_data, file, indent=4)

print(f"Cleaned data saved to {output_filename}")


Cleaned data saved to cleaned_data.json


In [20]:
import json
import pandas as pd
from datetime import datetime

# Upload the JSON file to Colab
from google.colab import files
uploaded = files.upload()

# Get the filename dynamically
filename = list(uploaded.keys())[0]

# Load the JSON file
with open(filename, 'r') as file:
    data = json.load(file)

# Convert it to a DataFrame
df = pd.json_normalize(data)

# Function to calculate age from DOB
def calculate_age(dob):
    if pd.isna(dob) or dob == '':
        return None
    dob = pd.to_datetime(dob, errors='coerce')
    if pd.isna(dob):
        return None
    today = datetime.today()
    age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    return age

# Add the 'age' column
df['age'] = df['patientDetails.birthDate'].apply(calculate_age)

# Define the function to categorize age into groups
def categorize_age(age):
    if pd.isna(age):
        return None
    if age <= 12:
        return 'Child'
    elif 13 <= age <= 19:
        return 'Teen'
    elif 20 <= age <= 59:
        return 'Adult'
    else:
        return 'Senior'

# Add the 'ageGroup' column
df['ageGroup'] = df['age'].apply(categorize_age)

# Count the number of 'Adult' patients
adult_count = df[df['ageGroup'] == 'Adult'].shape[0]

print(f"Count of Adult patients: {adult_count}")


Saving cleaned_data.json to cleaned_data (1).json
Count of Adult patients: 21


In [24]:
import json
import pandas as pd

# Upload the JSON file to Colab
from google.colab import files
uploaded = files.upload()

# Get the filename dynamically
filename = list(uploaded.keys())[0]

# Load the JSON file
with open(filename, 'r') as file:
    data = json.load(file)

# Convert the JSON data to a DataFrame
df = pd.json_normalize(data)

# Check if 'medicineName' column exists
if 'medicineName' in df.columns:
    # Count the occurrences of each medicineName
    medicine_counts = df['medicineName'].value_counts()

    # Get the 3rd most frequently prescribed medicine
    if len(medicine_counts) >= 3:
        third_most_common = medicine_counts.index[2]
        print(f"The 3rd most frequently prescribed medicine is: {third_most_common}")
    else:
        print("There are fewer than 3 different medicines in the dataset.")
else:
    print("Column 'medicineName' does not exist in the dataset.")


Saving DataEngineeringQ2.json to DataEngineeringQ2 (15).json
Column 'medicineName' does not exist in the dataset.


In [28]:
import json
from collections import Counter
from datetime import datetime

# Load the data from the JSON file
file_path = '/content/DataEngineeringQ2 (1).json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Helper function to calculate the percentage
def calculate_percentage(part, whole):
    return round((part / whole) * 100, 2)

# Extract patient details and medicines data
patient_details = [entry['patientDetails'] for entry in data]
medicines_data = [entry['consultationData']['medicines'] for entry in data]

# 1. Calculate percentage of missing values for firstName, lastName, and DOB
total_patients = len(patient_details)
missing_first_name = sum(1 for patient in patient_details if patient.get('firstName') in ["", None])
missing_last_name = sum(1 for patient in patient_details if patient.get('lastName') in ["", None])
missing_dob = sum(1 for patient in patient_details if patient.get('birthDate') in ["", None])

percent_missing_first_name = calculate_percentage(missing_first_name, total_patients)
percent_missing_last_name = calculate_percentage(missing_last_name, total_patients)
percent_missing_dob = calculate_percentage(missing_dob, total_patients)

# 2. Calculate the percentage of female gender after imputation
genders = [patient.get('gender') for patient in patient_details if patient.get('gender') not in ["", None]]
mode_gender = Counter(genders).most_common(1)[0][0]

# Impute missing gender values
imputed_genders = [patient.get('gender') if patient.get('gender') not in ["", None] else mode_gender for patient in patient_details]
percent_female_after_imputation = calculate_percentage(imputed_genders.count('F'), total_patients)

# 3. Add a column ageGroup and count the number of Adults
age_groups = []
current_date = datetime.now()

for patient in patient_details:
    birth_date_str = patient.get('birthDate')
    if birth_date_str not in ["", None]:
        birth_date = datetime.strptime(birth_date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        age = (current_date - birth_date).days // 365
        if 0 <= age <= 12:
            age_groups.append('Child')
        elif 13 <= age <= 19:
            age_groups.append('Teen')
        elif 20 <= age <= 59:
            age_groups.append('Adult')
        else:
            age_groups.append('Senior')
    else:
        age_groups.append(None)

adult_count = age_groups.count('Adult')

# 4. Calculate the average number of medicines prescribed
total_medicines_prescribed = sum(len(medicines) for medicines in medicines_data)
average_medicines_prescribed = round(total_medicines_prescribed / total_patients, 2)

# 5. Determine the 3rd most frequently prescribed medicineName
all_medicine_names = [medicine['medicineName'] for medicines in medicines_data for medicine in medicines]
medicine_name_counts = Counter(all_medicine_names)
third_most_frequent_medicine = medicine_name_counts.most_common(3)[-1][0]

# 6. Calculate the percentage distribution of active and inactive medicines
total_medicines = len(all_medicine_names)
active_medicines = sum(1 for medicines in medicines_data for medicine in medicines if medicine['isActive'])
inactive_medicines = total_medicines - active_medicines

percent_active_medicines = calculate_percentage(active_medicines, total_medicines)
percent_inactive_medicines = calculate_percentage(inactive_medicines, total_medicines)

{
    "percent_missing_first_name": percent_missing_first_name,
    "percent_missing_last_name": percent_missing_last_name,
    "percent_missing_dob": percent_missing_dob,
    "percent_female_after_imputation": percent_female_after_imputation,
    "adult_count": adult_count,
    "average_medicines_prescribed": average_medicines_prescribed,
    "third_most_frequent_medicine": third_most_frequent_medicine,
    "percent_active_medicines": percent_active_medicines,
    "percent_inactive_medicines": percent_inactive_medicines
}

{'percent_missing_first_name': 0.0,
 'percent_missing_last_name': 70.97,
 'percent_missing_dob': 32.26,
 'percent_female_after_imputation': 32.26,
 'adult_count': 21,
 'average_medicines_prescribed': 2.13,
 'third_most_frequent_medicine': 'C',
 'percent_active_medicines': 69.7,
 'percent_inactive_medicines': 30.3}